**Connecting to Google Drive to Access and Manipulate Dataset**





In [77]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/Idraak


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Idraak


In [78]:
%ls


dataSet/  dataSetssss/


**Importing the Required Libraries**

In [79]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [80]:
tf.__version__ 

'2.12.0'

**1. Data Preprocessing**

**Generation of Images for the Training Set**

In [81]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

**Generation of Images for the Testing Set**

In [82]:
test_datagen = ImageDataGenerator(rescale = 1./255)

**Creation of Training Set**

In [105]:
training_set = train_datagen.flow_from_directory('dataSet/trainingData',                                
                                                 target_size = (128, 128),
                                                 batch_size = 40,
                                                 color_mode = 'grayscale',                                
                                                 class_mode = 'categorical')

Found 18930 images belonging to 39 classes.


**Creation of Test Set**

In [106]:
test_set = test_datagen.flow_from_directory('dataSet/testingData',
                                            target_size = (128, 128),                                  
                                            batch_size = 40,        
                                            color_mode = 'grayscale',
                                            class_mode = 'categorical')

Found 8129 images belonging to 39 classes.


**2. Building the Convolutional Neural Network**

**Initialization of CNN**

In [107]:
classifier = tf.keras.models.Sequential()

**2.1. Convolution**

In [108]:
classifier.add(tf.keras.layers.Conv2D(filters=22,
                                     kernel_size=5, 
                                     padding="same", 
                                     activation="relu", 
                                     input_shape=[128, 128, 1]))

**2.2. Pooling**

In [109]:
classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

**--Adding of a Second Layer**

In [110]:
classifier.add(tf.keras.layers.Conv2D(filters=22, 
                                      kernel_size=5, 
                                      padding="same", 
                                      activation="relu"))

classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

In [99]:
# classifier.add(tf.keras.layers.Conv2D(filters=32, 
#                                       kernel_size=4, 
#                                       padding="same", 
#                                       activation="relu"))

# classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
#                                          strides=2, 
#                                          padding='valid'))

**2.3. Flattening**

In [111]:
classifier.add(tf.keras.layers.Flatten())

**2.4. Making Full Connection**

In [112]:
classifier.add(tf.keras.layers.Dense(units=128, 
                                     activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=96, activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=64, activation='relu'))
classifier.add(tf.keras.layers.Dense(units=39, activation='softmax')) 

**3. Training**

**Compilation of the CNN**

In [113]:
classifier.compile(optimizer = 'adam', 
                   loss = 'categorical_crossentropy', 
                   metrics = ['accuracy'])

**Training of the CNN on the Training set followed by Evaluation on the Test set**


In [114]:
classifier.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_21 (Conv2D)          (None, 128, 128, 22)      572       
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 64, 64, 22)       0         
 g2D)                                                            
                                                                 
 conv2d_22 (Conv2D)          (None, 64, 64, 22)        12122     
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 32, 32, 22)       0         
 g2D)                                                            
                                                                 
 flatten_8 (Flatten)         (None, 22528)             0         
                                                                 
 dense_28 (Dense)            (None, 128)              

In [115]:
classifier.fit(training_set,
                  epochs = 30,
                  validation_data = test_set)

Epoch 1/30
474/474 [==============================] - 3732s 8s/step - loss: 3.2157 - accuracy: 0.1181 - val_loss: 2.0683 - val_accuracy: 0.3561
Epoch 2/30
474/474 [==============================] - 827s 2s/step - loss: 1.3660 - accuracy: 0.5569 - val_loss: 1.7614 - val_accuracy: 0.5458
Epoch 3/30
474/474 [==============================] - 871s 2s/step - loss: 0.8687 - accuracy: 0.7083 - val_loss: 1.5132 - val_accuracy: 0.6151
Epoch 4/30
474/474 [==============================] - 875s 2s/step - loss: 0.6375 - accuracy: 0.7887 - val_loss: 1.7265 - val_accuracy: 0.6000
Epoch 5/30
474/474 [==============================] - 875s 2s/step - loss: 0.5013 - accuracy: 0.8343 - val_loss: 1.6807 - val_accuracy: 0.6177
Epoch 6/30
474/474 [==============================] - 872s 2s/step - loss: 0.4120 - accuracy: 0.8669 - val_loss: 1.8199 - val_accuracy: 0.6287
Epoch 7/30
474/474 [==============================] - 875s 2s/step - loss: 0.3595 - accuracy: 0.8820 - val_loss: 2.0129 - val_accuracy: 0.622

In [116]:
model_json = classifier.to_json()
with open("model_idraak.json", "w") as json_file:
    json_file.write(model_json)
print('Model Has Been Saved')
classifier.save_weights('model_idraak.h5')
print('Weights Have Been Saved')

Model Has Been Saved
Weights Have Been Saved
